# Intrusion Detection Computer Vision System

## Task 1 (Mandatory):
### Graphical Output
For each frame of the input video the system needs to show found blobs (either by coloring them on a black background or by showing the countours over the original video)
### Text Output
For each frame print the number of found objects, the value associated with each feature of the blob and its classification into person or other.

## Task 2 (Optional)
Develop an algorithm to distinguish between true objects and the removal of a previously present one.

## Video Characteristics
- 12 fps
- ~41s
- 320x240 pixels
- 8 bit/pixel (256 gray levels)

In [ ]:
# Imports

import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython import display

In [ ]:
# Global variables

input_video_path = "rilevamento-intrusioni-video.avi"
output_video_path = "test.avi"

In [ ]:
# Video Helper Functions

def play_video(video_path):
    '''
        Plays the video found in video_path frame by frame
    '''
    cap = cv2.VideoCapture(video_path)
    
    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret or frame is None:
                cap.release()
                print("Released Video Resource")
                break
            
            # Display frame
            plt.axis('off')
            plt.imshow(frame)
            plt.show()
            
            # Clear cell output when new frame is available
            display.clear_output(wait=True)
    except KeyboardInterrupt:
        cap.release()
        print("Released Video Resource")    

def edit_video(input_video_path, output_video_path, frame_transformation):
    '''
        Applies frame_transformation function to each frame taken from input_video_path and saves the result in
        output_video_path
    '''
    cap = cv2.VideoCapture(input_video_path)

    # Getting original video params
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (w,  h))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = frame_transformation(frame)
        # write the updated frame
        out.write(frame)
    cap.release()
    out.release()
    
def create_output_stream(cap, output_video_path):
    '''
        Saves cap in output_video_path
    '''
    # Getting original video params
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (w,  h))

    return out

In [ ]:
# Frame Editing Functions

def binarize_mask(mask, threshold=None):
    res = np.zeros(mask.shape)
    if mask.dtype == bool:
        res[mask] = 255
    else:
        res[mask < threshold] = 255
    return res

def gaussian_filter(frame, sigma=1.5, k_size=None):
    '''
        Higher sigmas should correspond to larger kernels. usually big as 
        Rule of thumb for a good kernel size given sigma
    '''
    if k_size is None:
        k_size = int(np.ceil((3*sigma))*2 + 1)
 
    return cv2.GaussianBlur(frame, (k_size,k_size) , sigma)

def preprocessing(frame, parameters):
    if parameters["preprocessing"] is not None:
        function, params = parameters["preprocessing"]
        return function(frame, *params)
    return frame

In [ ]:
# Distance Functions

def manhattan_distance(img1, img2):
    return np.sum(np.abs(img2 - img1), axis=-1)

def euclidean_distance(img1, img2):
    return np.sqrt(np.sum((img2 - img1) ** 2, axis=-1))

def maximum_distance(img1, img2):
    return np.max(img2 - img1, axis=-1)

In [ ]:
# Change Detection Functions

def two_frame_difference(prev_frame, curr_frame, d_func, threshold):
    return d_func(curr_frame, prev_frame) > threshold

def three_frame_difference(prev_frame, curr_frame, next_frame, d_func, threshold):
    diff1 = two_frame_difference(prev_frame, curr_frame, d_func, threshold)
    diff2 = two_frame_difference(curr_frame, next_frame, d_func, threshold)
    and_mask = np.prod([diff1, diff2],axis=0, dtype=bool)
    return and_mask

def background_subtraction(frame, background, d_func, threshold):
    frame = frame.astype(float)
    mask = d_func(frame, background) > threshold
    return mask

def background_set_initialization(input_video_path, parameter_set):
    bs = []
    for params in compute_parameters(parameter_set):
        cap = cv2.VideoCapture(input_video_path)
        bs.append({
            "image": background_initialization(cap, params["interpolation"], params["frames"]),
            "name": "{}_{}".format(params["frames"], params["interpolation"].__name__)
        })
    return bs
    

def background_initialization(cap, interpolation, n=100):
    # Loading Video
    bg = []
    idx = 0
    # Initialize the background image
    while(cap.isOpened() and idx < n):
        ret, frame = cap.read()
        if ret and not frame is None:
            frame = frame.astype(float)
            # Getting all first n images
            bg.append(frame)
            idx += 1
        else:
            break
    cap.release()

    bg_interpolated = np.stack(bg, axis=0)
    return interpolation(bg_interpolated, axis=0)

def initialize_subtractor():
    fgbg = cv2.createBackgroundSubtractorMOG2()
    fgbg.setDetectShadows(False)
    return fgbg

def background_subtraction_mog2(frame, fgbg):
    return fgbg.apply(frame)

def change_detection(frame, parameters):
    mask = background_subtraction_mog2(frame, parameters["subtractor"])
    #mask = background_subtraction(frame, parameters["background"]["image"], parameters["distance"], parameters["threshold"])
    return mask

In [ ]:
# Binary Morphology Function

def bm_test(mask):
    kernel1 = np.ones((3,3), np.uint8)
    kernel2 = np.ones((5,5), np.uint8)
    mask_int = mask.astype(np.uint8)

    mask_int = cv2.morphologyEx(mask_int, cv2.MORPH_OPEN, kernel1)
    mask_int = cv2.morphologyEx(mask_int, cv2.MORPH_CLOSE, kernel2)
    return mask_int.astype(bool)

def binary_morphology(mask, parameters):
    if parameters['morphology'] is None:
        return mask
    return parameters['morphology'](mask)

In [ ]:
import itertools

def prepare_output(mask):
    binary_mask = binarize_mask(mask)
    mask = np.tile(binary_mask[:,:,np.newaxis], 3)
    return np.uint8(mask)

def compute_parameters(param_set):
    return (dict(zip(param_set, x)) for x in itertools.product(*param_set.values()))

def generate_filename(directory, parameters):
    preprocessing_name = "none"
    if parameters["preprocessing"] is not None:
        function, params = parameters["preprocessing"]
        preprocessing_name = "_{}_{}".format(function.__name__, params)
       
    morphology_name = "none"
    if parameters["morphology"] is not None:
                morphology_name = parameters["morphology"].__name__

    
    return "{}tuning_{}_{}_{}_{}_{}.avi".format(
        directory, 
        parameters["background"]["name"],
        parameters["threshold"], 
        parameters["distance"].__name__,
        preprocessing_name,
        morphology_name
    )


In [ ]:
# Parameter Tuning

output_dir = "output/"

background_parameters_set = {
    "frames": [110, 130],
    "interpolation": [np.mean , np.median]
}

parameters_set = {
    "threshold": [35, 45],
    "distance": [euclidean_distance],
    "preprocessing": [None],
    "morphology": [bm_test],
    "background": background_set_initialization(input_video_path, background_parameters_set),
    "subtractor": [initialize_subtractor()]
}
                    
def compute_intrusion_detection(input_video_path, output_video_path, parameters):
    cap = cv2.VideoCapture(input_video_path)
    out = create_output_stream(cap, output_video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None:
            print("Computation finished! Video saved in {}".format(output_video_path))
            break
            
        preprocessed_frame = preprocessing(frame, parameters)
        mask_raw = change_detection(frame, parameters)
        mask_refined = binary_morphology(mask_raw, parameters)
        mask_output = prepare_output(mask_refined)
        out.write(mask_output)
    out.release()
    cap.release()

In [ ]:
# Execution

for parameters in compute_parameters(parameters_set):
    compute_intrusion_detection(input_video_path, generate_filename(output_dir, parameters), parameters)
print("Finished!")